In [1]:
from semantic_kernel import __version__
__version__

'1.39.0'

In [3]:
from semantic_kernel import Kernel
from semantic_kernel.connectors.ai.open_ai import AzureChatCompletion
from dotenv import load_dotenv
import os

load_dotenv()
kernel = Kernel()
service_id = "default"

kernel.add_service(
    AzureChatCompletion(service_id=service_id, 
                        deployment_name=os.getenv("DEPLOYMENT_NAME"),
                        endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
                        api_key=os.getenv("AZURE_OPENAI_API_KEY"))
)

In [11]:
### Adding Hugging face plugins

from semantic_kernel import Kernel
from semantic_kernel.connectors.ai.hugging_face import HuggingFaceTextCompletion, HuggingFaceTextEmbedding
from semantic_kernel.core_plugins import TextMemoryPlugin
from semantic_kernel.memory import SemanticTextMemory, VolatileMemoryStore

kernel = Kernel()
text_service_id = "HuggingFaceM4/tiny-random-LlamaForCausalLM"
kernel.add_service(
    service=HuggingFaceTextCompletion(
        service_id=text_service_id, ai_model_id=text_service_id, task="text-generation"
    ),
)
embed_service_id = "sentence-transformers/all-MiniLM-L6-v2"
embedding_svc = HuggingFaceTextEmbedding(service_id=embed_service_id, ai_model_id=embed_service_id)
kernel.add_service(
    service=embedding_svc,
)
memory = SemanticTextMemory(storage=VolatileMemoryStore(), embeddings_generator=embedding_svc)
kernel.add_plugin(TextMemoryPlugin(memory), "TextMemoryPlugin")


Microsoft Visual C++ Redistributable is not installed, this may lead to the DLL load failure.
It can be downloaded at https://aka.ms/vs/17/release/vc_redist.x64.exe


OSError: [WinError 126] The specified module could not be found. Error loading "C:\Users\CheekireddyDhaminiMA\AppData\Roaming\Python\Python311\site-packages\torch\lib\c10.dll" or one of its dependencies.

In [14]:
import os
import asyncio
from semantic_kernel import Kernel
from semantic_kernel.connectors.ai.open_ai import AzureChatCompletion, AzureTextEmbedding
from semantic_kernel.core_plugins import TextMemoryPlugin
from semantic_kernel.memory import SemanticTextMemory, VolatileMemoryStore
from semantic_kernel.prompt_template import PromptTemplateConfig
from semantic_kernel.connectors.ai.open_ai import AzureChatPromptExecutionSettings

kernel = Kernel()

# 1️⃣ Configure Azure OpenAI Services
chat_service_id = "azure_chat"
embed_service_id = "azure_embed"

# Add Chat Service (GPT)
kernel.add_service(
    AzureChatCompletion(
        service_id=chat_service_id,
        deployment_name="gpt-35-turbo", # Replace with your deployment name
        endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
        api_key=os.getenv("AZURE_OPENAI_API_KEY"),
    )
)

# Add Embedding Service (Ada)
embedding_svc = AzureTextEmbedding(
    service_id=embed_service_id,
    deployment_name="text-embedding-ada-002", # Replace with your deployment name
    endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
    api_key=os.getenv("AZURE_OPENAI_API_KEY"),
)
kernel.add_service(embedding_svc)

# 2️⃣ Configure Memory and the TextMemoryPlugin
# This connects the VolatileStore + Azure Embeddings to the Kernel
memory = SemanticTextMemory(storage=VolatileMemoryStore(), embeddings_generator=embedding_svc)
kernel.add_plugin(TextMemoryPlugin(memory), "TextMemoryPlugin")

async def main():
    # 3️⃣ Add Memories
    collection_id = "generic"
    await memory.save_information(collection=collection_id, id="info1", text="Sharks are fish.")
    await memory.save_information(collection=collection_id, id="info2", text="Whales are mammals.")
    await memory.save_information(collection=collection_id, id="info3", text="Penguins are birds.")
    
    print("✅ Memories saved to Volatile Store.")

    # 4️⃣ Define Prompt using {{recall}}
    # The 'recall' function looks into the memory automatically
    my_prompt = """
    I know these animal facts: 
    - {{recall 'sharks'}}
    - {{recall 'whales'}} 
    - {{recall 'penguins'}}

    Now, tell me something about: {{$request}}
    """.strip()

    execution_settings = AzureChatPromptExecutionSettings(
        service_id=chat_service_id,
        max_tokens=100,
        temperature=0.5,
    )

    prompt_template_config = PromptTemplateConfig(
        template=my_prompt,
        template_format="semantic-kernel",
        execution_settings=execution_settings,
    )

    my_function = kernel.add_function(
        function_name="animal_chat",
        plugin_name="AnimalPlugin",
        prompt_template_config=prompt_template_config,
    )

    # 5️⃣ Run the Chat
    user_input = "Compare sharks and whales."
    result = await kernel.invoke(my_function, request=user_input)
    
    print(f"\nUser: {user_input}")
    print(f"ChatBot:> {result}")

# Run
await main()

C:\Users\CheekireddyDhaminiMA\AppData\Local\Temp\ipykernel_19176\1906385738.py:37: DeprecationWarning: This class will be removed in a future version. Please use the InMemoryStore and Collection instead.
  memory = SemanticTextMemory(storage=VolatileMemoryStore(), embeddings_generator=embedding_svc)
C:\Users\CheekireddyDhaminiMA\AppData\Local\Temp\ipykernel_19176\1906385738.py:37: DeprecationWarning: This class will be removed in a future version.
  memory = SemanticTextMemory(storage=VolatileMemoryStore(), embeddings_generator=embedding_svc)
C:\Users\CheekireddyDhaminiMA\AppData\Local\Temp\ipykernel_19176\1906385738.py:38: DeprecationWarning: This class is deprecated and will be removed in a future version. Use the new `collection.as_text_search` instead.
  kernel.add_plugin(TextMemoryPlugin(memory), "TextMemoryPlugin")


✅ Memories saved to Volatile Store.

User: Compare sharks and whales.
ChatBot:> Certainly! Here’s a comparison between sharks and whales:

- **Classification**: Sharks are fish, specifically cartilaginous fish, meaning their skeletons are made of cartilage rather than bone. Whales are mammals, which means they are warm-blooded, breathe air through lungs, and nurse their young with milk.

- **Respiration**: Sharks breathe through gills, extracting oxygen from water. Whales breathe air using lungs and must come to the surface regularly to breathe through their blow


In [70]:
### Native funtions - a regular function, which is indicated using @kernel_function
## A Native function with multiple functions

from semantic_kernel.functions import kernel_function, KernelArguments

class WeatherPlugin:
    @kernel_function(
        name="get_weather",
        description="Gets the current weather report for a specific country"
    )
    def get_weather(self, city: str) -> str:
        if "London" in city:
            return "15 degrees and rainy."
        return "22 degrees and sunny"


##### Manul Invocation

In [ ]:
async def manual_invoke_function():
    city_input="London"
    kernel.add_plugin(WeatherPlugin(), plugin_name="Weather")
    result = await kernel.invoke(
        plugin_name="Weather", 
        function_name="get_weather",
        arguments=KernelArguments(city=city_input),
    )
    print("Result:  ",result)

await manual_invoke_function()  

#### Auto Invoke

In [79]:
from semantic_kernel.connectors.ai.function_choice_behavior import FunctionChoiceBehavior

async def auto_invoke_example():
    
    kernel.add_service(AzureChatCompletion(
        service_id="test_now",
        deployment_name="gpt-4.1",
        endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
        api_key=os.getenv("AZURE_OPENAI_API_KEY")
    ))
    settings = kernel.get_prompt_execution_settings_from_service_id("test_now")
    settings.function_choice_behavior = FunctionChoiceBehavior.Auto()

    user_query = "I'm going to London tomorrow. What's the weather like?"

    response = await kernel.invoke_prompt(
        prompt=user_query,
        settings=settings,
    )

    print("AI Response: ", response)

In [ ]:
await auto_invoke_example()

AI Response:  The weather in London tomorrow will be 15°C with rain. You may want to bring an umbrella or raincoat!


: 